In [1]:
import pandas as pd

all_results_df = pd.read_csv('../deep_learning/all_results.csv')

In [2]:
all_results_df.head()

,Unnamed: 0,unique_id,ds,NBEATS,NBEATSx,TCN,RNN,NHITS,MLP,LSTM,GRU,DilatedRNN,TFT,Informer,Autoformer,vintage_file
0,0,1,3/31/2018,0.007760,0.006407,0.007067,0.005208,0.005570,0.004725,0.004754,0.005907,0.006073,0.001064,0.005659,0.006359,vintage_2018_05
1,1,1,6/30/2018,0.007044,0.007414,0.007067,0.007846,0.007772,0.006252,0.009373,0.006786,0.007410,0.007691,0.005477,0.004458,vintage_2018_05
2,2,1,9/30/2018,0.002642,0.007136,0.007067,0.005208,0.005808,0.005107,0.005297,0.005547,0.005604,0.000361,0.005520,0.006603,vintage_2018_05
3,3,1,12/31/2018,0.008088,0.007688,0.007067,0.009935,0.007068,0.007151,0.013420,0.009762,0.011557,-0.003174,0.005348,0.002510,vintage_2018_05
4,4,1,3/31/2018,0.004881,0.006995,0.006402,0.008227,0.011824,0.005760,0.006207,0.005415,0.005824,0.001064,0.005398,0.003111,vintage_2018_06


In [3]:
all_results_df['ds'] = pd.to_datetime(all_results_df['ds'])

trel_dfs = []
for i in range(4):
    trel = all_results_df.iloc[i::4, :]
    trel = trel.drop(columns=['Unnamed: 0', 'unique_id'])
    trel_dfs.append(trel)

id_vars = ['ds', 'vintage_file']

tidy_trel_dfs = []
for trel_df in trel_dfs:
    tidy_trel = pd.melt(trel_df, id_vars=id_vars, var_name='model', value_name='value')
    tidy_trel_dfs.append(tidy_trel)

trel_values = [0, 1, 2, 3]
for trel_df, trel_value in zip(tidy_trel_dfs, trel_values):
    trel_df['trel'] = trel_value

combined_tidy_trel = pd.concat(tidy_trel_dfs, ignore_index=True)

# combined_tidy_trel['model'] = combined_tidy_trel['model'].str.replace('Auto', '')
combined_tidy_trel['vintage_file'] = combined_tidy_trel['vintage_file'].str.extract(r'(\d{4}_\d{2})')
combined_tidy_trel['vintage_file'] = combined_tidy_trel['vintage_file'].str.replace('_', '-')

In [4]:
combined_tidy_trel.head()

,ds,vintage_file,model,value,trel
0,2018-03-31,2018-05,NBEATS,0.007760,0
1,2018-03-31,2018-06,NBEATS,0.004881,0
2,2018-03-31,2018-07,NBEATS,0.005149,0
3,2018-06-30,2018-08,NBEATS,0.007181,0
4,2018-06-30,2018-09,NBEATS,0.007986,0


In [5]:
combined_tidy_trel.to_csv('../deep_learning/tidy_results.csv', index=False)